In [1]:
# get the data
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2023-03-27 14:33:58--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat.2’

train_32x32.mat.2   100%[===================>] 173.61M  16.3MB/s    in 14s     

2023-03-27 14:34:12 (12.7 MB/s) - ‘train_32x32.mat.2’ saved [182040794/182040794]

--2023-03-27 14:34:12--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat.2’

test_32x32.mat.2    100%[===================>]  61.30M  14.9MB/s    in 6.9s    

2023-03-27 14:34:19 (8.90 MB/s) - ‘test_32x32.mat.2’ saved [642

In [2]:
import numpy as np

In [3]:
# to convert .mat format into numpy array use the given code
import scipy.io
train_data = scipy.io.loadmat("train_32x32.mat")
X_train = train_data['X']
y_train = train_data['y']
test_data = scipy.io.loadmat("test_32x32.mat")
X_test = test_data['X']
y_test = test_data['y']

In [4]:
X_train.shape, X_test.shape

((32, 32, 3, 73257), (32, 32, 3, 26032))

In [5]:
d1, d2, d3, d4 = X_train.shape
X_train = X_train.reshape((d1*d2*d3, d4))
X_test = X_train.reshape((d1*d2*d3, d4))

In [6]:
X_train.shape, type(X_train)

((3072, 73257), numpy.ndarray)

In [7]:
# Normalize the images by dividing by 255

X_train = ((X_train/255).T).astype('float32')
X_test = ((X_test/255).T).astype('float32')
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

In [8]:
np.unique(y_train-1)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [9]:
y_train = y_train - 1
y_test = y_test - 1
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
X_train.shape

(73257, 3072)

In [11]:
from random import uniform
import numpy as np

#Xavier-initialization
n_inputs, n_hidden1, n_hidden2, n_outputs = (3072, 500, 100, 10)
W1 = np.array([[uniform(-1/4, 1/4) for i in range(n_inputs)] for i in range(n_hidden1)]).T
b1 = np.array([uniform(-1/4, 1/4) for i in range(n_hidden1)]).reshape(n_hidden1).T
W2 = np.array([[uniform(-1/4, 1/4) for i in range(n_hidden1)] for i in range(n_hidden2)]).T
b2 = np.array([uniform(-1/4, 1/4) for i in range(n_hidden2)]).reshape(n_hidden2).T
W3 = np.array([[uniform(-1/4, 1/4) for i in range(n_hidden2)] for i in range(n_outputs)]).T
b3 = np.array([uniform(-1/4, 1/4) for i in range(n_outputs)]).reshape(n_outputs).T
W1.shape, b1.shape

((3072, 500), (500,))

In [12]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience

In [13]:
# # X_batch = X_train[:batch_size, :]
# # # y_batch = y_train[:batch_size, :]
# # y_batch = y_train[:batch_size].squeeze()
# # X_batch.shape, y_batch.shape
# g = torch.Generator().manual_seed(2147483647) # for reproducibility

# ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
# Xb, Yb = X_train[ix], y_train[ix].squeeze() # batch X,Y
# Xb.shape, Yb.shape

In [14]:
rng = np.random.default_rng(seed=2147483647)
ix = rng.choice(X_train.shape[0], size=batch_size, replace=False)
Xb, Yb = X_train[ix], y_train[ix].squeeze() # batch X,Y
Xb.shape, Yb.shape

((32, 3072), (32,))

In [15]:
# # utility function we will use later when comparing manual gradients to PyTorch gradients
# def cmp(s, dt, t):
#   ex = torch.all(dt == t.grad).item()
#   app = torch.allclose(dt, t.grad)
#   maxdiff = (dt - t.grad).abs().max().item()
#   print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

# parameters = [W1, b1, W2, b2, W3, b3]
# print("number of parameters in total:",sum(p.nelement() for p in parameters)) # number of parameters in total
# for p in parameters:
#   p.requires_grad = True

In [16]:
parameters = [W1, b1, W2, b2, W3, b3]
total_params = sum(p.size for p in parameters)
print("number of parameters in total:", total_params) # number of parameters in total
for p in parameters:
    p.setflags(write=True)

number of parameters in total: 1587610


In [17]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def dsigmoid(x):
    # s = sigmoid(x)
    s = x 
    return s * (1.0 - s)

In [18]:
# # forward pass
# # Linear layer 1
# hpre1 = Xb @ W1 + b1 # hidden layer pre-activation
# # Non-linearity
# h1 = torch.tanh(hpre1) # hidden layerlogprobs

# # Linear layer 2
# hpre2 = h1 @ W2 + b2 # hidden layer pre-activation
# # Non-linearity
# h2 = torch.tanh(hpre2) # hidden layerlogprobs

# # Linear layer 3
# logits = h2 @ W3 + b3 # output layer

# # cross entropy loss (same as F.cross_entropy(logits, Yb))
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()
# loss

# forward pass
# Linear layer 1
hpre1 = Xb @ W1 + b1 # hidden layer pre-activation
# Non-linearity
# h1 = np.tanh(hpre1) # hidden post-activation
h1 = sigmoid(hpre1) # hidden post-activation

# Linear layer 2
hpre2 = h1 @ W2 + b2 # hidden layer pre-activation
# Non-linearity
# h2 = np.tanh(hpre2) # hidden post-activation
h2 = sigmoid(hpre2) # hidden post-activation

# Linear layer 3
logits = h2 @ W3 + b3 # output layer

# cross entropy loss
logit_maxes = np.max(logits, axis=1, keepdims=True)
norm_logits = logits - logit_maxes
counts = np.exp(norm_logits)
counts_sum = np.sum(counts, axis=1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = np.log(probs)
loss = -logprobs[np.arange(n), Yb].mean()
loss

2.4875401903104946

In [19]:
# # parameters = [W1, b1, W2, b2, W3, b3]
# # print("number of parameters in total:",sum(p.nelement() for p in parameters)) # number of parameters in total
# # for p in parameters:
# #   p.requires_grad = True

# # PyTorch backward pass
# for p in parameters:
#   p.grad = None
# for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
#           norm_logits, logit_maxes, logits, h2, hpre2, h1, hpre1]:
#   t.retain_grad()
# loss.backward()
# loss

In [20]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = np.zeros_like(logprobs)
dlogprobs[np.arange(n), Yb] = -1.0/n
dprobs = (1.0 / probs) * dlogprobs
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True)
dcounts = counts_sum_inv * dprobs
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
dcounts += np.ones_like(counts) * dcounts_sum
dnorm_logits = counts * dcounts
dlogits = dnorm_logits.copy()
dlogit_maxes = (-dnorm_logits).sum(1, keepdims=True)
dlogits += np.eye(logits.shape[1])[np.argmax(logits, axis=1)] * dlogit_maxes
dh2 = dlogits @ W3.T
dW3 = h2.T @ dlogits
db3 = dlogits.sum(0)
# dhpre2 = (1.0 - h2**2) * dh2
dhpre2 = dsigmoid(h2)
dh1 = dhpre2 @ W2.T
dW2 = h1.T @ dhpre2
db2 = dhpre2.sum(0)
# dhpre1 = (1.0 - h1**2) * dh1
dhpre1 = dsigmoid(h1)
dW1 = Xb.T @ dhpre1
db1 = dhpre1.sum(0)

# dlogprobs = torch.zeros_like(logprobs)
# dlogprobs[range(n), Yb] = -1.0/n
# dprobs = (1.0 / probs) * dlogprobs
# dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
# dcounts = counts_sum_inv * dprobs
# dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
# dcounts += torch.ones_like(counts) * dcounts_sum
# dnorm_logits = counts * dcounts
# dlogits = dnorm_logits.clone()
# dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
# dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
# dh2 = dlogits @ W3.T
# dW3 = h2.T @ dlogits
# db3 = dlogits.sum(0)
# dhpre2 = (1.0 - h2**2) * dh2
# dh1 = dhpre2 @ W2.T
# dW2 = h1.T @ dhpre2
# db2 = dhpre2.sum(0)
# dhpre1 = (1.0 - h1**2) * dh1
# dW1 = Xb.T @ dhpre1
# db1 = dhpre1.sum(0)

#------------------------------------------------
    
# cmp('logprobs', dlogprobs, logprobs)
# cmp('probs', dprobs, probs)
# cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
# cmp('counts_sum', dcounts_sum, counts_sum)
# cmp('counts', dcounts, counts)
# cmp('norm_logits', dnorm_logits, norm_logits)
# cmp('logit_maxes', dlogit_maxes, logit_maxes)
# cmp('logits', dlogits, logits)
# cmp('h2', dh2, h2)
# cmp('W3', dW3, W3)
# cmp('b3', db3, b3)
# cmp('hpre2', dhpre2, hpre2)
# cmp('h1', dh1, h1)
# cmp('W2', dW2, W2)
# cmp('b2', db2, b2)
# cmp('hpre1', dhpre1, hpre1)
# cmp('W1', dW1, W1)
# cmp('b1', db1, b1)


In [21]:
optimizer = "sgd"
# optimizer = "rmsprop"

max_steps = 200
batch_size = 32
n = batch_size # convenience
lossi = []

cache = [np.zeros_like(param) for param in parameters]
beta = 0.9
epsilon = 1e-8

# kick off optimization
for i in range(max_steps):

    # minibatch construct
    ix = rng.choice(X_train.shape[0], size=batch_size, replace=False)
    Xb, Yb = X_train[ix], y_train[ix].squeeze() # batch X,Y

    # forward pass
    # Linear layer 1
    hpre1 = Xb @ W1 + b1 # hidden layer pre-activation
    # Non-linearity
    h1 = sigmoid(hpre1) # hidden post-activation

    # Linear layer 2
    hpre2 = h1 @ W2 + b2 # hidden layer pre-activation
    # Non-linearity
    h2 = sigmoid(hpre2) # hidden post-activation

    # Linear layer 3
    logits = h2 @ W3 + b3 # output layer

    # cross entropy loss
    logit_maxes = np.max(logits, axis=1, keepdims=True)
    norm_logits = logits - logit_maxes
    counts = np.exp(norm_logits)
    counts_sum = np.sum(counts, axis=1, keepdims=True)
    counts_sum_inv = counts_sum**-1
    probs = counts * counts_sum_inv
    logprobs = np.log(probs)
    loss = -logprobs[np.arange(n), Yb].mean()

    # # backward pass

    # manual backprop! #swole_doge_meme
    # -----------------
    dlogprobs = np.zeros_like(logprobs)
    dlogprobs[np.arange(n), Yb] = -1.0/n
    dprobs = (1.0 / probs) * dlogprobs
    dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True)
    dcounts = counts_sum_inv * dprobs
    dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
    dcounts += np.ones_like(counts) * dcounts_sum
    dnorm_logits = counts * dcounts
    dlogits = dnorm_logits.copy()
    dlogit_maxes = (-dnorm_logits).sum(1, keepdims=True)
    dlogits += np.eye(logits.shape[1])[np.argmax(logits, axis=1)] * dlogit_maxes
    dh2 = dlogits @ W3.T
    dW3 = h2.T @ dlogits
    db3 = dlogits.sum(0)
    dhpre2 = dsigmoid(h2)
    dh1 = dhpre2 @ W2.T
    dW2 = h1.T @ dhpre2
    db2 = dhpre2.sum(0)
    dhpre1 = dsigmoid(h1)
    dW1 = Xb.T @ dhpre1
    db1 = dhpre1.sum(0)
    grads = [dW1, db1, dW2, db2, dW3, db3]
    # -----------------

    # update
    lr = 0.1 if i < 100 else 0.01 # step learning rate decay
    if optimizer=="sgd":
      # Stochastic Gradient Descent (SGD)
      for p, grad in zip(parameters, grads):
        #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
        p += -lr * grad # new way of swole doge TODO: enable
      
    if optimizer == "rmsprop":
        for p, grad, c in zip(parameters, grads, cache):
            c = beta * c + (1 - beta) * grad ** 2
            p += - lr * grad / (np.sqrt(c) + epsilon)

    # track stats
    if i % 20 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:1d}: {loss:.4f}')
    lossi.append(np.log10(loss.item()))

  #   if i >= 100: # TODO: delete early breaking when you're ready to train the full net
  #     break

      0/200: 2.4297


<ipython-input-17-2f07b60c5344>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


     20/200: 2.3100
     40/200: 2.2587
     60/200: 2.2826
     80/200: 2.2115
    100/200: 2.3225
    120/200: 2.3190
    140/200: 2.3161
    160/200: 2.2731
    180/200: 2.2953


In [22]:
def f1_score(y_true, y_pred):
    # y_pred = np.array(y_pred_probs.argmax(1))
    # y_true = y_true.ravel()
    # get unique labels
    y_true = y_true.squeeze()
    # print(y_pred.shape, y_true.shape)
    labels = np.unique(np.concatenate((y_true, y_pred)))
    # print(np.unique(y_pred))
    # initialize variables to store precision, recall, and f1 score
    precision = np.zeros(len(labels))
    recall = np.zeros(len(labels))
    f1 = np.zeros(len(labels))
    
    # iterate over each label
    for i, label in enumerate(labels):
        # calculate true positives, false positives, and false negatives
        tp = np.sum((y_true == label) & (y_pred == label))
        fp = np.sum((y_true != label) & (y_pred == label))
        fn = np.sum((y_true == label) & (y_pred != label))
        
        # calculate precision, recall, and f1 score
        if tp == 0:
            precision[i] = 0
            recall[i] = 0
            f1[i] = 0
        else:
            precision[i] = tp / (tp + fp)
            recall[i] = tp / (tp + fn)
            f1[i] = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])
    
    # weighted average of f1 score
    f1_score = np.sum(f1) / len(labels)
    return f1_score

In [23]:
# evaluate train and test loss
probs=None
def split_loss(split):
    x,y = {
      'train': (X_train, y_train),
      # 'val': (X_val, Ydev),
      'test': (X_test, y_test),
    }[split]
    # forward pass
    # Linear layer 1
    hpre1 = x @ W1 + b1 # hidden layer pre-activation
    # Non-linearity
    h1 = np.tanh(hpre1) # hidden post-activation

    # Linear layer 2
    hpre2 = h1 @ W2 + b2 # hidden layer pre-activation
    # Non-linearity
    h2 = np.tanh(hpre2) # hidden post-activation

    # Linear layer 3
    logits = h2 @ W3 + b3 # output layer

    # cross entropy loss
    logit_maxes = np.max(logits, axis=1, keepdims=True)
    norm_logits = logits - logit_maxes
    counts = np.exp(norm_logits)
    counts_sum = np.sum(counts, axis=1, keepdims=True)
    counts_sum_inv = counts_sum**-1
    # global probs
    probs = counts * counts_sum_inv
    logprobs = np.log(probs)
    loss = -logprobs[np.arange(n), y].mean()
    print(split+" loss:", loss.item())

    if split=="test":
        # make predictions
        preds = np.argmax(probs, axis=1)

        # calculate accuracy
        acc = np.mean(preds == y_test)
        print("accuracy:", acc)

        # calculate f1 score
        f1 = f1_score(y_train, preds)
        print("f1 score:", f1)

split_loss('train')
split_loss('test')

train loss: 2.982219565960013
test loss: 2.9425288784721384
accuracy: 0.1958743085433313
f1 score: 0.03182120801671297
